# Worked Example: Thomas Algorithm for 1D Heat Conduction


In this notebook, we implement the **Thomas Algorithm** (also known as the **TDMA** – Tridiagonal Matrix Algorithm)
to solve a system of linear equations arising from the discretization of the 1D steady-state heat conduction equation.

We assume:
- Constant thermal conductivity
- No internal heat generation
- Dirichlet boundary conditions

The governing differential equation is:

$$
\frac{d^2 T}{dx^2} = 0
$$

This is discretized using second-order finite differences on a uniform grid.



## Finite Difference Discretization

Suppose we have a rod of length \( L \) divided into \( N+2 \) nodes (including boundaries), so there are \( N \) internal nodes.
The discretized equation at internal node \( i \) becomes:

$$
- T_{i-1} + 2T_i - T_{i+1} = 0 \quad \text{for } i = 1, 2, ..., N
$$

This results in a tridiagonal system:

$$
a_i T_{i-1} + b_i T_i + c_i T_{i+1} = d_i
$$

where:
- \( a_i = -1 \)
- \( b_i = 2 \)
- \( c_i = -1 \)
- \( d_i = 0 \) (modified for boundary conditions)



## Thomas Algorithm

The Thomas algorithm is an efficient way to solve tridiagonal systems. It consists of two steps:

1. **Forward Elimination**:
   Modify the coefficients to eliminate the lower diagonal.
2. **Backward Substitution**:
   Solve for the solution vector from the last equation to the first.

This reduces the computational complexity to \( \mathcal{O}(N) \).


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def thomas_algorithm(a, b, c, d):
    n = len(d)
    # Forward elimination
    for i in range(1, n):
        w = a[i] / b[i - 1]
        b[i] = b[i] - w * c[i - 1]
        d[i] = d[i] - w * d[i - 1]

    # Back substitution
    x = np.zeros(n)
    x[-1] = d[-1] / b[-1]
    for i in reversed(range(n - 1)):
        x[i] = (d[i] - c[i] * x[i + 1]) / b[i]

    return x



## Worked Example

Let’s solve the 1D heat conduction problem on a rod of length \( L = 1 \) with the following boundary conditions:

- \( T(0) = 100^\circ C \)
- \( T(1) = 200^\circ C \)

We divide the domain into \( N = 5 \) internal nodes. The system becomes:

$$
A \vec{T} = \vec{d}
$$

We construct the tridiagonal system accordingly.


In [ ]:

# Parameters
N = 5
T0 = 100
TL = 200

# Tridiagonal coefficients
a = np.full(N, -1.0)
b = np.full(N, 2.0)
c = np.full(N, -1.0)
d = np.zeros(N)

# Modify RHS for boundary conditions
d[0] += T0
d[-1] += TL

# Solve
T_internal = thomas_algorithm(a, b, c, d)

# Full temperature vector
T_full = np.concatenate(([T0], T_internal, [TL]))
x = np.linspace(0, 1, N + 2)

# Plot
plt.plot(x, T_full, marker='o')
plt.xlabel('x')
plt.ylabel('Temperature (°C)')
plt.title('1D Steady-State Heat Conduction')
plt.grid(True)
plt.show()



## Conclusion

We used the **Thomas Algorithm** to efficiently solve a system of equations resulting from the finite difference discretization
of the 1D steady-state heat conduction equation. The results match the expected linear temperature profile.
